<a href="https://colab.research.google.com/github/schmellerl/gradient_flows_order_parameters_mechanics/blob/main/colab/Three_Phase_pureDef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/ColabNumerics/Example3_pureDef/'

In [ ]:
!git clone https://github.com/schmellerl/gradient_flows_order_parameters_mechanics.git

from gradient_flows_order_parameters_mechanics.colab.Data.refine_mesh_pureDef   import refine_mesh_pureDef 
from gradient_flows_order_parameters_mechanics.colab.Data.Neumann_construction  import Neumann_construction
from gradient_flows_order_parameters_mechanics.colab.Data.model_parameters      import *
from gradient_flows_order_parameters_mechanics.colab.Data.postprocessing_pureDef import *

<h1>Gradient flows for coupling order parameters and mechanics</h1>

FENICS implementation of the examples from

Schmeller, L. & Peschka, D. (2022). 


DOI: http://dx.doi.org/10.20347/WIAS.PREPRINT.2909

<h3>General coupled gradient flow evolution</h3>

For a state variable $q=(u,\psi)$ with displacement $u:\Omega\to\mathbb{R}^d$ and phase field $\psi:\Omega\to\mathbb{R}^N$ we consider the following free energy 
\begin{align}
    \mathscr{F}(q) 
    = & 
    \int_{{\Omega}}W_{\rm elast}(F_e,\psi){\rm d}x
    + \int_{{\Omega}}W_{\rm phase}(\psi,\nabla\psi,F)
    {\rm d}x
\end{align}

for deformation gradient $F=\mathbb{I}_d+\nabla u=F_e F_p$ and with given plastic strain $F_p=F_p(\psi)$ and $F_e=FF_p^{-1}$.

We consider a Neo-Hookean elastic energy density 
\begin{align}
   W_{\rm elast}(F_e,\psi) = \frac{G}{2}\left(\text{tr}(F_e^\top F_e - \mathbb{I}_d) -2\log(\det(F_e))\right)+
 \frac{K}{2}\big(\det(F_e)-H\big)^2  
\end{align}
with bulk modulus $G=G(\psi)$ and phase volume $H=H(\psi)$ and inverse compressibility $K\in\mathbb{R}$. The remaining part of the free energy is 
\begin{align}
    W_\text{phase}(\psi,\nabla\psi,F) &= \left[\frac{1}{2}\nabla\psi\cdot \sigma\nabla\psi + W_\text{entropy}(\psi,F)\right]\det(F)
\end{align}
with second-order tensor $\sigma=\sigma(\psi,F)\in\mathbb{R}^{d\times d}$. 
Together with a given dissipation potential $R(q,\dot{q})$, different parameters, double-well or Flory-Huggins-type entropy $W_\text{entropy}$ and possible constraints $C(q,\nabla q)=0\in\mathbb{R}^M$ added through the Lagrangian in terms of $q_\lambda=(q,\lambda)$ with the optional Lagrange multiplier $\lambda:\Omega\to\mathbb{R}^M$
\begin{align}
\mathscr{L}(q_\lambda)=\mathscr{F}(q)+\int_\Omega C(q,\nabla q)\cdot\lambda\,{\rm d}x
\end{align}
we consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$, which we solve by a saddle-point problem generated by the minimization problem
\begin{align}
%\partial_t q =-\nabla_R \mathscr{L}(q)
%\quad\Leftrightarrow\quad \left(
  \min_{v_\lambda=(v,\hat{\lambda})} \Big[R(q,v)+\langle \mathrm{D}\mathscr{L}(q_\lambda),v_\lambda\rangle\Big]%\right)
\end{align}
solved by incremental minimization as described in more detail in the manuscript.

In [ ]:
from fenics import *
from matplotlib import pyplot as plt
import numpy as np
from mshr import *

# Function spaces
mesh    = Mesh('/content/gradient_flows_order_parameters_mechanics/colab/Data/mesh.xml')
mesh1   = Mesh('/content/gradient_flows_order_parameters_mechanics/colab/Data/mesh1.xml')

Vu_solid  = VectorElement("P", mesh1.ufl_cell(), 1)
V_solid   = FunctionSpace(mesh1,Vu_solid)

P2        = VectorElement("P", mesh.ufl_cell(), 1)
P1        = FiniteElement("P", mesh.ufl_cell(), 1)
R         = FiniteElement("P", mesh.ufl_cell(), 1)    

TH        = MixedElement([P2,R]) 
W         = FunctionSpace(mesh, TH)
S         = FunctionSpace(mesh, P1)

# homogeneous Dirichlet boundary conditions 
bc = [DirichletBC(W.sub(0),  Constant((0, 0)), 'on_boundary')]

def GshearF(psi1,psi2):
    psi3 = -1-psi1-psi2
    xi1 = (1+psi1)/2
    xi2 = (1+psi2)/2
    xi3 = (1+psi3)/2
    return  (xi1*G1 + xi2*G2 + xi3*G3)

In [ ]:
def incremental_minimization(old_q, dt):
    
    q, dq  = Function(W), TestFunction(W)   
    v, lambda1   = split(q)       # Current solution
    old_v,_      = split(old_q)   # Old solution
    dv,_         = split(dq)      # Test functions

    # Define elasticity stuff
    d       = v.geometric_dimension() 
    I       = Identity(d)
    F       = grad(v) + I  # strain
    J       = det(F)       # Jacobian
    C       = F.T*F        # (right) Cauchy-Green tensor    

    psi11 = Expression(("tanh(ff*(h0-x[1])/eps)"),eps=eps,h0=h0,ff=interface_factor,degree=2)
    psi22 = Expression(("-1 + 2*(1+tanh(ff*(x[1]/h0-h0)/eps))*(1+tanh(ff/eps*(h1-pow(pow((x[0])/h0-H1/2,2.0)+pow(x[1]/h0-h0,2.0),0.5))))/4"),eps=eps,h0=h0,h1=h1,H1=H1,ff=interface_factor,degree=2)    
    
    psi1 = interpolate(psi11, S)
    psi2 = interpolate(psi22, S)
    
    psi3 = project(-1-psi1-psi2,S)    
   
    gradpsi1 = inv(F).T*grad(psi1) # 
    gradpsi2 = inv(F).T*grad(psi2) # 
    gradpsi3 = inv(F).T*grad(psi3) # 

    # Stored strain energy density (compressible neo-Hookean model)
    e_elastic   = (GshearF(psi1,psi2)/2)*tr(C - I)

    e_phase    = gamma1*( 1/(4*eps)*(1-psi1**2)**2 + (eps/2)*inner(gradpsi1,gradpsi1) )
    e_phase   += gamma2*( 1/(4*eps)*(1-psi2**2)**2 + (eps/2)*inner(gradpsi2,gradpsi2) )
    e_phase   += gamma3*( 1/(4*eps)*(1-psi3**2)**2 + (eps/2)*inner(gradpsi3,gradpsi3) )

    E   = e_elastic*dx + e_phase*dx  
    L   = E + lambda1*(J-1)*dx

    # Finite difference time derivatives
    dot_v  = (v-old_v)/dt
 
    # Build residual:
    Res   = derivative(L, q, dq) 
    
    # Evolution 
    Res   += inner(2*mu*sym(grad(dot_v)) , sym(grad(dv))  )*dx  

    # Solve
    q.assign(old_q)  
    solve(Res==0,q,bc)
    E_free  = assemble(E)

    return q,E_free

In [ ]:
# initial conditions
initial = Expression(("0","0","0"), degree = 2) 

old_q = interpolate(initial, W)

In [ ]:
t = 0
energies = []

bc, W, S, P2, mesh, psi1, psi2  = refine_mesh_pureDef(mesh,W,S,old_q,eps,h0,h1,interface_factor,H1)
old_q                           = interpolate(initial, W)
bc, W, S, P2, mesh, psi1, psi2  = refine_mesh_pureDef(mesh,W,S,old_q,eps,h0,h1,interface_factor,H1)
old_q                           = interpolate(initial, W)
bc, W, S, P2, mesh, psi1, psi2  = refine_mesh_pureDef(mesh,W,S,old_q,eps,h0,h1,interface_factor,H1)
old_q                           = interpolate(initial, W)
bc, W, S, P2, mesh, psi1, psi2  = refine_mesh_pureDef(mesh,W,S,old_q,eps,h0,h1,interface_factor,H1)
old_q                           = interpolate(initial, W)

In [ ]:
t         = 0
n_steps   = 300
dt        = 1e-3
energies  = []

output_mesh(mesh,filepath)
output_mesh1(mesh1,filepath)
output_solution(old_q,0,mesh,P2,psi1,psi2,filepath,t)


out   = display(progress(0, n_steps), display_id=True)
for n in range(n_steps):
  dt_adapt = dt
  if (n<10) :
    dt_adapt = dt/10
  if (n>9) :
    dt_adapt = dt 

  t += dt
  q,E_free = incremental_minimization(old_q, dt)  
  out.update(progress(n+1,n_steps))
  energies.append([n,t,E_free])
  output_solution(q,n+1,mesh,P2,psi1,psi2,filepath,t)
  old_q.assign(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)

**Neumann constcution**

In [ ]:
def Neumann_construction(theta_in_liquid, theta_in_solid, gamma_sa ):

  theta_L = theta_in_liquid*pi/180   
  theta_S = theta_in_solid*pi/180

  theta_2 = 180*pi/180 - theta_S
  theta_1 = theta_L - theta_2

  gamma_la = gamma_sa * 1/(cos(theta_1)+cos(theta_2)*sin(theta_1)/sin(theta_2) )
  gamma_sl = sin(theta_1)/sin(theta_2) * gamma_la

  gamma_s = (gamma_sa + gamma_sl - gamma_la)/2
  gamma_l = gamma_sl - gamma_s 
  gamma_a = gamma_sa - gamma_s
  
  return gamma_s, gamma_l, gamma_a

In [ ]:
# Shoud be the same as the parameters from model_parameters.py
gamma_sa          = 23e-3     #N/m
theta_in_liquid   = 68   
theta_in_solid    = 114 

gamma_s, gamma_l, gamma_a = Neumann_construction(68, 114, gamma_sa )

G_shear = 1e3
H_DIM      = 1e-5 

gamma1 = gamma_s/(G_shear*H_DIM)
gamma2 = gamma_l/(G_shear*H_DIM)
gamma3 = gamma_a/(G_shear*H_DIM)

print('Neumann construction surface tension: ', gamma1,gamma2,gamma3)